In [1]:
!pip install byol_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from byol_pytorch import BYOL
from torchvision import models
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


# Load the CIFAR 10 dataset and create a data loader for it
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Load the pretrained ResNet model and create a BYOL learner
resnet = models.resnet50(pretrained=True)
learner = BYOL(
    resnet,
    image_size=32,
    hidden_layer='avgpool'
).to('cuda')

# Train the BYOL learner on the CIFAR 10 dataset
opt = torch.optim.Adam(learner.parameters(), lr=3e-4)
for epoch in range(10):
    for i, data in enumerate(trainloader, 0):
        images, labels = data
        images, labels= images.to('cuda'), labels.to('cuda')
        loss = learner(images)
        opt.zero_grad()
        loss.backward()
        opt.step()
        learner.update_moving_average() # update moving average of target encoder
    print("epoch count =>", epoch)
  
torch.save(resnet.state_dict(), './improved-net.pt')

# Extract features from the BYOL learner for each image in the CIFAR 10 dataset
features = []
targets = []
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        features.append(resnet(images).to('cpu').numpy())
        targets.append(labels.numpy())
features = np.concatenate(features)
targets = np.concatenate(targets)

# Build a KNN model using the extracted features and the labels of the CIFAR 10 dataset
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(features, targets)

# Evaluate the KNN model on a test set
test_features = []
test_targets = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        test_features.append(learner.encoder(images).numpy())
        test_targets.append(labels.numpy())
test_features = np.concatenate(test_features)
test_targets = np.concatenate(test_targets)
accuracy = knn.score(test_features, test_targets)
print("KNN accuracy on test set:", accuracy)


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommend

epoch count => 0
epoch count => 1
epoch count => 2
epoch count => 3
epoch count => 4
epoch count => 5
epoch count => 6
epoch count => 7
epoch count => 8
epoch count => 9


RuntimeError: ignored

In [35]:
import torchvision
import torch.nn as nn
resnet_model = torchvision.models.resnet50(pretrained=False)
# Extract features from the BYOL learner for each image in the CIFAR 10 dataset
resnet_model.load_state_dict(torch.load('improved-net.pt', map_location='cuda'))
resnet_model= resnet_model.to('cuda')

resnet_model.fc=nn.Identity()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [41]:



features = []
targets = []
resnet_model.eval()
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images,labels= images.to('cuda'), labels.to('cuda')
        feature=resnet_model(images)
        print(type(feature))
        features.append(feature.cpu().numpy())
        targets.append(labels.cpu().numpy())
features = np.concatenate(features)
targets = np.concatenate(targets)

# Build a KNN model using the extracted features and the labels of the CIFAR 10 dataset
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(features, targets)

# Evaluate the KNN model on a test set
test_features = []
test_targets = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images,labels= images.to('cuda'), labels.to('cuda')
        test_features.append(resnet_model(images).cpu().numpy())
        test_targets.append(labels.cpu().numpy())
test_features = np.concatenate(test_features)
test_targets = np.concatenate(test_targets)
accuracy = knn.score(test_features, test_targets)
print("KNN accuracy on test set:", accuracy)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

TypeError: ignored